# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [97]:
# Dependencies and Setup
import numpy as np
import pandas as pd
import hvplot.pandas
import requests
import geopandas as gpd
import geoviews as gv
import geoviews.feature as gf
from geoviews import dim

# Import API key
from api_keys import geoapify_key

In [98]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tarariras,-34.1618,-57.4928,60.04,76,91,9.04,UY,1687446000
1,1,sittwe,20.2850,93.4857,75.58,98,100,3.00,MM,1687498200
2,2,cacoal,-11.5763,-61.6890,90.21,42,0,4.56,BR,1687449600
3,3,homs,34.6863,36.7023,91.38,22,0,18.28,SY,1687510800
4,4,fuding,27.7232,119.8090,69.66,99,100,3.87,CN,1687492800


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [121]:
%%capture --no-display
# Convert the DataFrame to a geoviews Dataset
dataset = gv.Dataset(city_data_df, kdims=['Lng', 'Lat'], vdims=['Humidity'])

# Define the points on the map with the size determined by the humidity
points = dataset.to(gv.Points, ['Lng', 'Lat'], ['Humidity']).opts(size=dim('Humidity')*0.1)

# Create the map and overlay the points on it
world_map = gf.land().opts(color='white', line_color='black')
map_with_points = world_map * points

# Display the map
map_with_points.opts(width=800, height=500)


:Overlay
   .Land.I   :Feature   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [140]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temps_df = city_data_df[(city_data_df['Max Temp'] >= 75) & (city_data_df['Max Temp'] <= 80) & (city_data_df['Cloudiness'] <= 70)]

# Drop any rows with null values
ideal_temps_df.dropna()

# Display sample data
ideal_temps_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,maun,-21.0913,22.9880,79.34,22,0,13.18,BW,1687514400
11,11,orumiyeh,37.5226,45.4382,78.69,51,0,22.44,IR,1687509000
19,19,chebba,35.1616,11.3752,78.66,67,0,9.31,TN,1687518000
63,63,zhanaozen,44.0212,53.6247,79.59,14,0,16.67,KZ,1687507200
65,65,cinisi,38.8504,13.0291,77.18,84,2,10.42,IT,1687518000


### Step 3: Create a new DataFrame called `hotel_df`.

In [143]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,tarariras,UY,-34.1618,-57.4928,76,
1,sittwe,MM,20.2850,93.4857,98,
2,cacoal,BR,-11.5763,-61.6890,42,
3,homs,SY,34.6863,36.7023,22,
4,fuding,CN,27.7232,119.8090,99,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [154]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "filter": "",
    "bias": "",
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{latitude},{longitude},{radius}"
    params["bias"] = f"proximity:{latitude},{longitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    resp = requests.get(base_url, params=params)

    # print('resp')
    # print(resp.url)
    
    # Convert the API response to JSON format
    name_address = resp.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tarariras - nearest hotel: No hotel found
sittwe - nearest hotel: No hotel found
cacoal - nearest hotel: No hotel found
homs - nearest hotel: No hotel found
fuding - nearest hotel: No hotel found
nampula - nearest hotel: No hotel found
maun - nearest hotel: No hotel found
puerto lempira - nearest hotel: No hotel found
port nelson - nearest hotel: No hotel found
olonkinbyen - nearest hotel: No hotel found
tuburan - nearest hotel: No hotel found
orumiyeh - nearest hotel: No hotel found
tommot - nearest hotel: No hotel found
sishilichengzi - nearest hotel: No hotel found
kaa-khem - nearest hotel: No hotel found
marabba - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
ola - nearest hotel: No hotel found
nyurba - nearest hotel: No hotel found
chebba - nearest hotel: No hotel found
oban - nearest hotel: No hotel found
spring creek - nearest hotel: No hotel found
bomi - nearest hotel: No hotel found
galesong - nearest hotel: No hotel found
vyaz

,City,Country,Lat,Lng,Humidity,Hotel Name
0,tarariras,UY,-34.1618,-57.4928,76,No hotel found
1,sittwe,MM,20.2850,93.4857,98,No hotel found
2,cacoal,BR,-11.5763,-61.6890,42,No hotel found
3,homs,SY,34.6863,36.7023,22,No hotel found
4,fuding,CN,27.7232,119.8090,99,No hotel found
...,...,...,...,...,...,...
303,mayuge,UG,0.1341,33.5755,82,No hotel found
304,muana,BR,-0.8372,-49.2498,66,No hotel found
305,alajarvi,FI,63.2352,23.7857,57,No hotel found
306,dalandzadgad,MN,44.3706,104.6890,19,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [155]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE